In [1]:
!pip install pytesseract
!pip install PyPDF2

In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract
from tqdm import tqdm
from pytesseract import Output
from PIL import Image
from PyPDF2 import PdfReader

In [7]:
# im = Image.open("train_dataset/receipt_00503.png")
# im.show()

In [8]:
# extract text from .pdf to dictionary
def create_df_pdf(path_to_pdfs='pdf_docs/'):
    dict_df_pdf = {}
    for pdf in tqdm(os.listdir(path_to_pdfs)):
        path_to_pdf = path_to_pdfs+pdf
        reader = PdfReader(path_to_pdf)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        dict_df_pdf[path_to_pdf] = text
    
    return dict_df_pdf

In [9]:
new_df_pdf = create_df_pdf()

100%|█████████████████████████████████████████| 177/177 [00:11<00:00, 15.17it/s]


In [10]:
data_frame = pd.DataFrame(new_df_pdf, index=range(0, len(new_df_pdf))).T.iloc[:, 1]
data_frame.head()

pdf_docs/7202208265.pdf    Customer: First Canadian Diamond Cutting Works...
pdf_docs/Page-49.pdf       TYPE\nTYPE\nPRODUCT DESCRIPTION\nPRODUCT DESCR...
pdf_docs/220220826.pdf     Customer: First Canadian Diamond Cutting Works...
pdf_docs/8202208263.pdf    Customer: First Canadian Diamond Cutting Works...
pdf_docs/7202208263.pdf    Customer: First Canadian Diamond Cutting Works...
Name: 1, dtype: object

In [11]:

with open('all_pdf_text.txt', 'w') as f:
    for index, line in tqdm(zip(data_frame.index, data_frame.values)):
        f.write(index)
        f.write('\n'+line +'\n')

177it [00:00, 89932.38it/s]


### **************************************************

In [12]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() 
f = open('annotations(1).json')
TRAIN_DATA = json.load(f)

2022-09-12 11:34:22.536947: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:34:22.575027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:34:22.575763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [13]:
TRAIN_DATA

{'classes': ['BILL_TO', 'SHIP_TO', 'INVOICE_N', 'DATE', 'TOTAL'],
 'annotations': [["pdf_docs/7202208265.pdf\nCustomer: First Canadian Diamond Cutting Works Inc. Ship To: First Canadian Diamond Cutting Works Inc.\nATTN: Duty Deffered\nAddress: 4058 Jean Talon W., Suite 100 4058 Jean Talon W., Suite 100\nMontreal, Quebec Montreal, Quebec\nCanada Canada\nH4P 1V5 H4P 1V5\nInvoice: '7202208265 Date: 2022-08-26  Silver Market:  $21.35\nGOLD IN ADVANCE, BALANCE PAY C.O.D.\nPO NO. STYLE DESCRIPTION UNIT WT/PC @PRICE PRICE PER AMT(USD)\nPO493449 FC171H-4HPL 10PC Silver White DIAMOND Ring $66,7867 667,8674\nSilver Metal 3.85 38.51 WT 0.63 2.45 24.45\n12% Silver Loss Loss 0.46 4.62 WT 0.63 0.29 2.93\nSilver Metal 0WT 0.63 0\n12% Silver Loss Loss 0WT 0.63 0\n5 Diamond DIAMOND 0.06 0.63 wt 277.88 17.4 174.04\n13 Diamond DIAMOND 0.05 0.52 wt 188.56 9.87 98.75\n6 Diamond DIAMOND 0.02 0.24 wt 188.56 4.57 45.65\n24 Diamond DIAMOND 0.07 0.71 wt 188.56 13.4 133.98\n4 Diamond DIAMOND 0.03 0.33 wt 317.58 

In [14]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|████████████████████████████████████████| 117/117 [00:00<00:00, 123.53it/s]


In [15]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2022-09-12 11:34:36.994462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:34:36.998635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:34:36.998894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [16]:
# if you want to use gpu add "--gpu-id [index-gpu]"

!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2022-09-12 11:34:42.712693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:34:42.716749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 11:34:42.716996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-09-12 11:34:44,189] [INFO] Set up nlp object from config
[2022-09-12 11:34:44,196] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-09-12 11:34:44,199] [INFO] Created vocabulary
[2022-09-12 11:34:44,199] [INFO] Finishe

In [17]:
nlp_ner = spacy.load("model-best/") 

In [18]:
for pdf in data_frame:
    doc = nlp_ner(pdf)
    spacy.displacy.render(doc, style="ent", jupyter=True)
    

In [32]:
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - ' +str(ent.start_char) + ' - ' + str(ent.end_char) +
                ' - ' + ent.label_ + ' - ' + str(spacy.explain(ent.label_)))
            print('\n')
    else:
        print('No named entities found.')

In [33]:
def pdf_ner(path_to_pdf):
    reader = PdfReader(path_to_pdf)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    doc = nlp_ner(text)
    show_ents(doc)
#     spacy.displacy.render(doc, style="ent", jupyter=True)

In [34]:
pdf_ner('pdf_docs/120220823.pdf')

Ship To: - 35 - 43 - SHIP_TO - None


Address: 11083 Route 9 - 69 - 91 - BILL_TO - None


11083 Route 9 - 93 - 106 - SHIP_TO - None


Champlain, New York - 108 - 127 - BILL_TO - None


Champlain, New York - 128 - 147 - SHIP_TO - None


United States - 148 - 161 - BILL_TO - None


United States - 162 - 175 - SHIP_TO - None


12919 - 176 - 181 - BILL_TO - None


12919 - 182 - 187 - SHIP_TO - None


Invoice: '120220823 - 188 - 207 - INVOICE_N - None


Date: 2022-08-22 - 208 - 224 - DATE - Absolute or relative dates or periods


USD: 20658.3244 - 3391 - 3406 - TOTAL - None


Total: $20658,32 - 3805 - 3821 - TOTAL - None




### tesseract

In [77]:

# Adding custom options
custom_config = r'--oem 3 --psm 6'
# pytesseract.image_to_string(img, config=custom_config)

In [78]:
def get_tesseract_ocr():
    ocr_for_each_receipt = {}
    for receipt in tqdm(os.listdir('train_dataset/')):
        path_to_img = "train_dataset/"+receipt
        img = cv2.imread(path_to_img)
        norm_img = np.zeros((img.shape[0], img.shape[1]))
        img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
        img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)[1]
        img = cv2.GaussianBlur(img, (1, 1), 0)
        ocr_for_each_receipt[path_to_img] = pytesseract.image_to_string(img, lang='eng', config=custom_config)
    return ocr_for_each_receipt

In [79]:
 pytesseract.image_to_string("train_dataset/receipt_00000.png", lang='eng', config=custom_config)

'i. ™\nKg\nmM  &£€:\nJ “= 7 "\n\x0c'

In [80]:
tesseract_ocr_df = get_tesseract_ocr()

100%|█████████████████████████████████████████| 800/800 [14:35<00:00,  1.09s/it]


In [81]:
tesseract_ocr_df

{'train_dataset/receipt_00503.png': 'ES\nwo! 1\nIx 22727 Re22727\n\n, Ayan Grg Kalasan\nPi 3x 4545 Rp13635\nTempe\nHOSE ST Rp36362\nTAX ST + 36362 |\nPB I RP3636\nDIFFER Re2 /\nITEMS 40 CHE\nCASH *. 40000\nFe ee ttre:\n10m aie\na |\n| ly ae ep ee /\n\x0c',
 'train_dataset/receipt_00296.png': 'a a\n\ns\n\nmo ~ 6\n\nwy Be th 1a f\n\n: ALMOND CREAM CHEESE\n2. 2 x 28,000 56.060\nf°" SUBTOTAL 56.000\n4 GRANDTOTAL 56.000\n: CASH IDR 56 000\nCHANGE DUE 3\nxy OF\n\x0c',
 'train_dataset/receipt_00706.png': '1 SIRSAK 10, 000\n\n1 SUNKIST 10, 006\n2 aT\n\nTOTAL 20, 000\n\nCASH 20, 000\n\x0c',
 'train_dataset/receipt_00285.png': '1 Danan Torrens ae\n~ MRO Salt (MJ 40, O00\n, Mineral aater (3) 10, 006\nranta StwOry (L) 25. 900\nAT \\\n\nPUT AL Cope\nCASH oy\nCHANGED\n\x0c',
 'train_dataset/receipt_00630.png': 'y\nore +e :\ni Grilled Saison Head 4 Sng\ni Set Menu Faaily 332,505\ni Tairyos Salmon Neki “a6 000\n2 Tepanyaki Cawaniuchi aq°oce = |\nod Teppah Seafood Fr Re 36i000 |?\n__h Tobtke Gunkon 24,

In [82]:
data_frame_tess = pd.DataFrame(tesseract_ocr_df, index=range(0, len(new_df_ocr))).T.iloc[:, 1]
data_frame_tess.head()

train_dataset/receipt_00503.png    ES\nwo! 1\nIx 22727 Re22727\n\n, Ayan Grg Kala...
train_dataset/receipt_00296.png    a a\n\ns\n\nmo ~ 6\n\nwy Be th 1a f\n\n: ALMON...
train_dataset/receipt_00706.png    1 SIRSAK 10, 000\n\n1 SUNKIST 10, 006\n2 aT\n\...
train_dataset/receipt_00285.png    1 Danan Torrens ae\n~ MRO Salt (MJ 40, O00\n, ...
train_dataset/receipt_00630.png    y\nore +e :\ni Grilled Saison Head 4 Sng\ni Se...
Name: 1, dtype: object

In [83]:
data_frame_tess.to_csv('saved_data_tess.csv')

## easy ocr test

In [36]:
# !pip install easyocr

In [3]:
import easyocr

In [4]:
def create_easy_ocr():
    reader = easyocr.Reader(['en'])
    easy_ocr_trans = {}
    for receipt in tqdm(os.listdir('train_dataset/')):
        path_to_img = "train_dataset/"+receipt
        full_text = ' '
        for line in reader.readtext(path_to_img, detail = 0, paragraph=False):
            full_text += line
            full_text += ' '
        easy_ocr_trans[path_to_img] = full_text
    return easy_ocr_trans

In [1]:
# new_df_ocr = create_easy_ocr()

In [31]:
data_frame = pd.DataFrame(new_df_ocr, index=range(0, len(new_df_ocr))).T.iloc[:, 1]
data_frame.head()

train_dataset/receipt_00503.png     Cted Ix 22727 Re22727 Ayam Grg Kalasan 3x 454...
train_dataset/receipt_00296.png     Laa /0 T ALMOND CREAM CHEESE 2 X 28,000 56,00...
train_dataset/receipt_00706.png     Aede SIRSAK 10 ,000 SUNK I ST 10,000 2 QT TOT...
train_dataset/receipt_00285.png     Salt (H) 40.00o Hinera] Water ` (8) 10.00o Fa...
train_dataset/receipt_00630.png     1 Grilled Salmon Head 24,500 1 Set Menu Famil...
Name: 1, dtype: object

In [35]:
data_frame.to_csv("saved_data.csv")

In [89]:
data_frame.index

Index(['train_dataset/receipt_00503.png', 'train_dataset/receipt_00296.png',
       'train_dataset/receipt_00706.png', 'train_dataset/receipt_00285.png',
       'train_dataset/receipt_00630.png', 'train_dataset/receipt_00732.png',
       'train_dataset/receipt_00226.png', 'train_dataset/receipt_00071.png',
       'train_dataset/receipt_00675.png', 'train_dataset/receipt_00146.png',
       ...
       'train_dataset/receipt_00132.png', 'train_dataset/receipt_00670.png',
       'train_dataset/receipt_00707.png', 'train_dataset/receipt_00776.png',
       'train_dataset/receipt_00133.png', 'train_dataset/receipt_00247.png',
       'train_dataset/receipt_00299.png', 'train_dataset/receipt_00257.png',
       'train_dataset/receipt_00715.png', 'train_dataset/receipt_00413.png'],
      dtype='object', length=800)

In [104]:
with open('all_ocr_text.txt', 'w') as f:
    for index, line in tqdm(zip(data_frame.index, data_frame.values)):
        f.write(index)
        f.write('\n'+line +'\n')
        f.write('\n')

800it [00:00, 395689.06it/s]
